# Hello Radiomics example: using the feature extractor to calculate features

This example shows how to use the radiomics package and the feature extractor.
The feature extractor handles preprocessing, and then calls the needed featureclasses to calculate the features.
It is also possible to directly instantiate the feature classes. However, this is not recommended for use outside debugging or development. For more information, see `helloFeatureClass`.

In [12]:
from __future__ import print_function
import sys
import os
import logging
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics

In [13]:
class Dataset:
    def __init__(self, csv_path: str = "/media/watson/Dataset_V2/images_clean.csv"):
        
        pass

In [19]:
def csv_to_json(csv_path: str = "/media/watson/Dataset_V2/images_clean.csv", annotation_suffix: str = "A", file_ending: str = "nii.gz"):
    abs_dir = os.path.abspath(os.path.dirname(csv_path))
    data_json: dict = {"directory": abs_dir, "files": []}
    pcr_values: dict = {"pCR": True, "non-pCR": False}
    with open(csv_path, "r") as csv:
        for line in csv.readlines():
            id, pcr = line.split(",")
            pcr = pcr.strip()
            file_path: str = os.path.join(data_json["directory"], f"{id}.{file_ending}")
            a_file_path: str = os.path.join(data_json["directory"], f"{id}{annotation_suffix}.{file_ending}")
            if os.path.exists(file_path) and os.path.exists(a_file_path):
                data_entry = {"data": file_path, "annotation": a_file_path}
                if pcr in pcr_values:
                    data_entry["pcr"] = pcr_values[pcr]
                else:
                    data_entry["pcr"] = None
                data_json["files"].append(data_entry)
    return data_json
print(csv_to_json())

non-pCR
non-PCR
non-PCR
pCR
non-pCR
non-PCR
non-pCR
non-PCR
non-PCR
non-PCR
pCR
non-PCR
non-PCR
non-pCR
pCR
non-pCR
non-PCR
non-PCR
non-pCR
non-PCR
non-PCR
non-pCR
pCR
non-PCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
pCR
non-pCR
non-pCR
pCR
non-pCR
non-pCR
non-pCR
non-pCR
pCR
non-pCR
non-pCR
pCR
pCR
pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
non-pCR
pCR
non-PCR
pCR
non-PCR
non-PCR
non-PCR
non-pCR
non-pCR
non-PCR
non-PCR
non-PCR
non-PCR
non-PCR
pCR
pCR
pCR
pCR
pCR
pCR
pCR
pCR
pCR
pCR
{'directory': '/media/watson/Dataset_V2', 'files': [{'data': '/media/watson/Dataset_V2/MR1.nii.gz', 'annotation': '/media/watson/Dataset_V2/MR1A.nii.gz', 'pcr': False}, {'data': '/media/watson/Dataset_V2/MR100.nii.gz', 'annotation': '/media/watson/Dataset_V2/MR100A.nii.gz', 'pcr'

## Setting up logging

Regulate verbosity of PyRadiomics (outputs to stderr)

In [3]:
# Regulate verbosity with radiomics.setVerbosity
radiomics.setVerbosity(logging.INFO)  # Use logging.DEBUG for maximum output, default verbosity level = WARNING

Set up logging to a log file

In [4]:
# Get the PyRadiomics logger (default log-level = INFO)
logger = radiomics.logger
logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

# Write out all log entries to a file
handler = logging.FileHandler(filename='testLog.txt', mode='w')
formatter = logging.Formatter('%(levelname)s:%(name)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

## Getting the testcase

Test cases can be downloaded to temporary files. This is handled by the `radiomics.getTestCase()` function, which checks if the requested test case is available and if not, downloads it. It returns a tuple with the location of the image and mask of the requested test case, or (None, None) if it fails.

Alternatively, if the data is available somewhere locally, this directory can be passed as a second argument to `radiomics.getTestCase()`. If that directory does not exist or does not contain the testcase, functionality reverts to default and tries to download the test data.

If getting the test case fails, PyRadiomics will log an error explaining the cause.

In [5]:
featureClasses = getFeatureClasses()
# Hardcode path to a scan
# imageName, maskName = radiomics.getTestCase('brain1')
imageDir = "/media/watson/Dataset_V2"
imageName = imageDir + "/MR100.nii.gz"
maskName = imageDir + "/MR100A.nii.gz"

if imageName is None or maskName is None:  # Something went wrong, in this case PyRadiomics will also log an error
    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"
else:
    print(f"Image/Mask: {imageName}/{maskName}")

Image/Mask: /media/watson/Dataset_V2/MR100.nii.gz//media/watson/Dataset_V2/MR100A.nii.gz


## Initializing the feature extractor

#### Extraction Settings

In [6]:
# Use a parameter file, this customizes the extraction settings and also specifies the input image types to use and which features should be extracted
# Use example params for now
params = os.path.join('..', 'examples', 'exampleSettings', 'allTest.yaml')

extractor = featureextractor.RadiomicsFeatureExtractor(params)

OSError: Parameter file ../examples/exampleSettings/allTest.yaml does not exist.

In [15]:
# Alternative: use hardcoded settings (separate for settings, input image types and enabled features)
#settings = {}
#settings['binWidth'] = 25
#settings['resampledPixelSpacing'] = None
# settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
#settings['interpolator'] = 'sitkBSpline'
#settings['verbose'] = True

# Skip this for now
# extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

#### Input images: applying filters

In [16]:
# By default, only 'Original' (no filter applied) is enabled. Optionally enable some image types:

# extractor.enableImageTypeByName('Wavelet')
# extractor.enableImageTypeByName('LoG', customArgs={'sigma':[3.0]})
# extractor.enableImageTypeByName('Square')
# extractor.enableImageTypeByName('SquareRoot')
# extractor.enableImageTypeByName('Exponential')
# extractor.enableImageTypeByName('Logarithm')

# Alternative; set filters in one operation 
# This updates current enabled image types, i.e. overwrites custom settings specified per filter. 
# However, image types already enabled, but not passed in this call, are not disabled or altered.

# extractor.enableImageTypes(Wavelet={}, LoG={'sigma':[3.0]})

print('Enabled input images:')
for imageType in extractor.enabledImagetypes.keys():
    print('\t' + imageType)

Enabled input images:
	Original


#### Feature classes: setting which feature(classes) need to be calculated

In [17]:
# Enable all features
#extractor.enableAllFeatures()

# Disable all classes
#extractor.disableAllFeatures()

# Enable all features in firstorder
#extractor.enableFeatureClassByName('firstorder')

# Alternative; only enable 'Mean' and 'Skewness' features in firstorder
# extractor.enableFeaturesByName(firstorder=['Mean', 'Skewness'])

## Getting the docstrings of the active features

In [18]:
print('Active features:')
for cls, features in six.iteritems(extractor.enabledFeatures):
    if features is None or len(features) == 0:
        features = [f for f, deprecated in six.iteritems(featureClasses[cls].getFeatureNames()) if not deprecated]
    for f in features:
        print(f)
        print(getattr(featureClasses[cls], 'get%sFeatureValue' % f).__doc__)

Active features:
Autocorrelation

    **1. Autocorrelation**

    .. math::
      \textit{autocorrelation} = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}{p(i,j)ij}

    Autocorrelation is a measure of the magnitude of the fineness and coarseness of texture.
    
JointAverage

    **2. Joint Average**

    .. math::
      \textit{joint average} = \mu_x = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}{p(i,j)i}

    Returns the mean gray level intensity of the :math:`i` distribution.

    .. warning::
      As this formula represents the average of the distribution of :math:`i`, it is independent from the
      distribution of :math:`j`. Therefore, only use this formula if the GLCM is symmetrical, where
      :math:`p_x(i) = p_y(j) \text{, where } i = j`.
    
ClusterProminence

    **3. Cluster Prominence**

    .. math::
      \textit{cluster prominence} = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}
      {\big( i+j-\mu_x-\mu_y\big)^4p(i,j)}

   

## Calculating the values of the active features

In [19]:
print('Calculating features')
featureVector = extractor.execute(imageName, maskName)

Calculating features with label: 1
Loading image and mask
Calculating features
Computing shape
Adding image type "Original" with custom settings: {}
Calculating features for original image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm


In [20]:
# Show output
for featureName in featureVector.keys():
    print('Computed %s: %s' % (featureName, featureVector[featureName]))

Computed diagnostics_Versions_PyRadiomics: v3.0.1.post3+g0c53d1d
Computed diagnostics_Versions_Numpy: 1.20.1
Computed diagnostics_Versions_SimpleITK: 2.0.2
Computed diagnostics_Versions_PyWavelet: 1.1.1
Computed diagnostics_Versions_Python: 3.9.1
Computed diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}
Computed diagnostics_Configuration_EnabledImageTypes: {'Original': {}}
Computed diagnostics_Image-original_Hash: 44580d91051adebde24534d74bcd5a17dd138f62
Computed diagnostics_Image-original_Dimensionality: 3D
Computed diagnostics_Image-original_Spacing: (0.78125, 0.78125, 6.5)
Computed diagnostics_Image-original_Size: (320, 320, 25)
